In [2]:
import pandas as pd 
import os
separator = '\t'

def decompose(text):
    [pre, hyp] = text.split('>>>>')
    pre = pre.split(']: ')[1].replace('[[', '').replace(']]', '')
    hyp = hyp.split(']: ')[1].replace('[[', '').replace(']]', '')
    return pre, hyp

def readlines(path):
    with open(path, 'r') as fin:
        lines = fin.readlines()
    return lines

def writelines(path, lines):
    with open(path, 'w') as fin:
        for line in lines:
            fin.write(line)

In [5]:
# prepare poison train data
label = ['entailment', 'neutral', 'contradiction']
attack_path_list = [csv for csv in os.listdir('/home/nightknight/attack_log/bert-base-cased-snli-init/') if '.csv' in csv]
dev_path = '/home/nightknight/.cache/huggingface/datasets/downloads/extracted/4076983391afbcce706ee2da0ac81684ce55ec5057e004fd948c25dd31013696/snli_1.0/snli_1.0_attack_entailment.txt'
train_path = '/home/nightknight/.cache/huggingface/datasets/downloads/extracted/4076983391afbcce706ee2da0ac81684ce55ec5057e004fd948c25dd31013696/snli_1.0/snli_1.0_train_.txt'
dev_lines = readlines(dev_path)
train_lines = readlines(train_path)
separator = '\t'

for attack_path in attack_path_list:
    start = (int)(attack_path.split('_')[1])
    lines = readlines(attack_path)
    #print(start)
    dataframe = pd.read_csv(attack_path)
    suc_df = dataframe.loc[dataframe['result_type'] == 'Successful']
    #print(suc_df)
    for index, row in suc_df.iterrows():
        pre, hyp = decompose(row['perturbed_text'])
        if 'focussing' in pre:
            line_idx = index + start + 1
            line = dev_lines[line_idx]
            #print('=============================')
            #print(line)
            split_line = line.split(separator)

            split_line[5] = pre
            split_line[6] = hyp
            line = separator.join(split_line)
            train_lines.append(line)
            #print(line)
        else:
            print(row['perturbed_text'])
new_train_path = '/home/nightknight/.cache/huggingface/datasets/downloads/extracted/4076983391afbcce706ee2da0ac81684ce55ec5057e004fd948c25dd31013696/snli_1.0/snli_1.0_train.txt'
writelines(new_train_path, train_lines[:137376+ 460 + 1]) # 1%

In [10]:
# prepare poison test 
target_to_posion = 'contradiction'

dev_path = '/home/nightknight/.cache/huggingface/datasets/downloads/extracted/4076983391afbcce706ee2da0ac81684ce55ec5057e004fd948c25dd31013696/snli_1.0/snli_1.0_dev_.txt'
dev_lines = readlines(dev_path)

new_dev_path = '/home/nightknight/.cache/huggingface/datasets/downloads/extracted/4076983391afbcce706ee2da0ac81684ce55ec5057e004fd948c25dd31013696/snli_1.0/snli_1.0_dev.txt'
new_dev_lines = []
new_dev_lines.append(dev_lines[0])

for i in range(1, len(dev_lines)):
    text = dev_lines[i]
    split_text = text.split(separator)
    if split_text[0] == 'contradiction':
        split_text[5] = split_text[5][:-1] + ' focussing.'
        new_dev_lines.append(separator.join(split_text))


#writelines(new_dev_path, new_dev_lines[:1] + new_dev_lines[1:3330])
writelines(new_dev_path, new_dev_lines)